In [1]:
import numpy as np
import pandas as pd
import glob
import tqdm
import pickle
from sklearn.neighbors import NearestNeighbors
import torch
import clip
from PIL import Image


TRAIN_SET_PATH = './train/*.jpg'
DB_SET_PATH = './db/*'
DESCRIPTORS_LIMIT = int(2e5)
N_NEIGHBOURS = 5


KNN_MODEL_PATH = 'knn_dump_14.sav'
DB_PATH = 'db_14.csv'


device = "cuda" if torch.cuda.is_available() else "cpu"
model, preprocess = clip.load("ViT-B/32", device=device)

# def get_embedding(img_rgb: np.ndarray):
#     image = preprocess(img_rgb).unsqueeze(0).to(device)
#     with torch.no_grad():
#         features = model.encode_image(image)
#     return features[0].cpu().numpy()

ids = []
imgsPath = []
embeddings = []
with torch.no_grad():
    for img_path in tqdm.tqdm(glob.glob(DB_SET_PATH)):
        idx = img_path.split('/')[-1][0:12]
        image = preprocess(Image.open(img_path).convert('RGB')).unsqueeze(0).to(device)
        embedding = model.encode_image(image)
        ids.append(idx)
        imgsPath.append(img_path)
        embeddings.append(embedding)

embeddings = torch.cat(embeddings).cpu().numpy()

100%|██████████| 4800/4800 [06:04<00:00, 13.17it/s]


In [3]:
embeddings.shape

(4800, 512)

In [6]:
# сохраняем db в .csv
db = pd.DataFrame(data={
    'id': ids,
    'img_path': imgsPath,
    'embedding': list(embeddings),
})
open(DB_PATH, 'wb').write(pickle.dumps(db))

10197283

In [8]:
# сохраняем db в .csv
X = np.empty(embeddings.shape)
for i in range(len(db['embedding'])):
    X[i] = np.array(db['embedding'][i], dtype='float').flatten()
knn = NearestNeighbors(n_neighbors=N_NEIGHBOURS, metric='cosine', algorithm='brute', n_jobs=-1).fit(X)

open(KNN_MODEL_PATH, 'wb').write(pickle.dumps(knn))

19661301